## Setup

In [1]:
import sys
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.spatial.distance import cdist
from google.colab.patches import cv2_imshow

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Cropping original frames

In [4]:
CLASS_INDEX = 0
X_INDEX = 1
Y_INDEX = 2
PERSON_CLASS = '0'
BALL_CLASS = '32'
X_SIZE = 1928
Y_SIZE = 1024
CROP_SIZE = 224 # pixels, size of crop = (CROP_SIZExCROP_SIZE)
radius = int(CROP_SIZE / 2)
ANY_INDEX = 0

In [5]:
# from os import listdir
# from os.path import isfile, join
# files = [file for file in listdir(FOLDER_FRAMES) if isfile(join(FOLDER_FRAMES, file))] # get a list of files in the directory
# filenames = [os.path.splitext(filename)[0] for filename in files] # separate filenames from their extension .jpg

## Test: doing above by iterating over 2 different folders (frames and labels)

In [6]:
POSITIVES_FOLDER = '/content/drive/MyDrive/Bundesliga/PositiveFrames/'
FOLDER_LABELS = POSITIVES_FOLDER + 'detect/'
FOLDER_FRAMES = POSITIVES_FOLDER + 'frames_ms2/'

FIRST_FRAMES = FOLDER_FRAMES + '1606b0e6_0/'
SECOND_FRAMES = FOLDER_FRAMES + '1606b0e6_1/'

In [7]:
CURRENT_FOLDER = POSITIVES_FOLDER
os.chdir(CURRENT_FOLDER)
! pwd

/content/drive/MyDrive/Bundesliga/PositiveFrames


## Main code:

In [8]:
# video = "_".join(filenames[ANY_INDEX].split("_", 2)[:2])
# rest = ''.join(filenames[ANY_INDEX].split("_", 2)[2:])
# y = filenames[ANY_INDEX].split("_")[3]
# video, rest, y

In [9]:
# current_labels = FOLDER_LABELS + video + '/labels/'
# current_labels

In [10]:
USE_PERSONS = True
USE_PERSONS_THRESH = 15
FIRST_FRAME_USED = 18
FRAMES_PER_EVENT = 35
NUM_CLASSES = 3
CONFIDENCE_INDEX = 5

FRAMES_FORMAT = 'VIDEOID_EVENTID_CLASSID_ORDERID.jpg'

Class meanings = {0: 'challenge', 1: 'throw-in', 2: 'play'}

In [11]:
# Challenges' frame between tolerances are respectively: 17 yes, 1 yes, 17 yes.
# For play and throw-in are respectively: 9 no, 8 yes, 1 yes, 8 yes, 9 no.

Implementation:

In [42]:
video_ids = ['1606b0e6_0', 
             '1606b0e6_1', # 274 samples, less than actual
             'cfbe2e94_0', 
             'cfbe2e94_1',
             'ecf251d4_0',
             '3c993bd2_0', 
             '3c993bd2_1', # 274 samples, less than actual
             '4ffd5986_0', 
             '9a97dae4_1', 
             '35bd9041_0', # 274 samples, less than actual
             '35bd9041_1',
             '407c5a9e_1',
             ]

Folders with missing frames (as discussed on Slack on 12/11/22):
- ['1606b0e6_1', '35bd9041_0', '3c993bd2_0']

In [32]:
EVENTS_PER_VIDEO = {
    '1606b0e6_0': 396,
    '1606b0e6_1': 274,
    'cfbe2e94_0': 305,
    'cfbe2e94_1': 285,
    'ecf251d4_0': 386,
    '3c993bd2_1': 274,
    '3c993bd2_1': 376,
    '4ffd5986_0': 302,
    '9a97dae4_1': 291,
    '35bd9041_0': 274,
    '35bd9041_1': 359,
    '407c5a9e_1': 350,
    }
    
# MAYBE CODE IT IN A BETTER WAY

In [14]:
POSITIVES_FOLDER = '/content/drive/MyDrive/Bundesliga/PositiveFrames/'
FOLDER_LABELS = POSITIVES_FOLDER + 'detect/'
FOLDER_FRAMES = POSITIVES_FOLDER + 'frames_ms2/'

FIRST_FRAMES = FOLDER_FRAMES + '1606b0e6_0/'
SECOND_FRAMES = FOLDER_FRAMES + '1606b0e6_1/'

In [15]:
os.chdir(POSITIVES_FOLDER)
! pwd

/content/drive/MyDrive/Bundesliga/PositiveFrames


In [35]:
videos = [video for video in EVENTS_PER_VIDEO.keys()]
videos

['1606b0e6_0',
 'cfbe2e94_0',
 'cfbe2e94_1',
 'ecf251d4_0',
 '3c993bd2_1',
 '4ffd5986_0',
 '9a97dae4_1',
 '35bd9041_1',
 '407c5a9e_1']

## Main code:

### Hyperparameter tunning to find best threshold for distance ball-person

In [20]:
! pwd

/content/drive/MyDrive/Bundesliga/PositiveFrames


In [21]:
video = '1606b0e6_0'
event = 0
class_ = 0
r = 1

In [22]:
from scipy import spatial

In [ ]:
event_id = f'{video}_{event}_{class_}'

frame_id = f'{event_id}_{FIRST_FRAME_USED - 1 + r}' # -1 so we get main fraim (index 17)
label_txt = f'{frame_id}.txt'
with open(f'{FOLDER_LABELS}{video}/labels/{label_txt}') as f:
  reader = f.read()
detections = [i.split(' ') for i in reader.split('\n')][:-1] # [:-1] because last element is always "''"

arr = np.array(detections)
persons = arr[arr[:, 0] == '0']
xy_persons = persons[:, X_INDEX:Y_INDEX + 1].astype(float)
detection = arr[arr[:, 0] == '32'] # DON'T USE IN ACTUAL CODE, YOU ALREADY HAVE 'DETECTION'
xy_ball = detection[:, X_INDEX:Y_INDEX + 1].astype(float)
xy_closest_person = xy_persons[spatial.KDTree(xy_persons).query(xy_ball)[1]] # get closest point
distance, index = spatial.KDTree(xy_persons).query(xy_ball) # get distance

frame_id, distance, xy_ball

### Understanding with code above what YOLO is seeing:

- distance 1606b0e6_0_0_0_17: 0.01715389 # ball confused with football boot
- distance 1606b0e6_0_1_0_17: 0.02588498 # ball behind goalkeeper
- distance 1606b0e6_0_2_1_17: 0.05881961 # ball behind goalkeeper, and 0.13542185 ball on court's side
- distance 1606b0e6_0_3_2_20: 0.01331324 # BALL ON PLAY

- distance 1606b0e6_0_4_2_19: 0.05413774 # BALL ON PLAY

- distance 1606b0e6_0_5_2_17: 0.22416412 # ball on top right court's side
- distance 1606b0e6_0_5_2_19: 0.22460785 # ball on top right court's side
- distance 1606b0e6_0_5_2_20: 0.22483156 # ball on top right court's side
- distance 1606b0e6_0_5_2_21: 0.22660578 # ball on top right court's side, and 0.01887572 BALL ON PLAY

- distance 1606b0e6_0_6_2_17: 0.21772145 # ball on top right court's side
- distance 1606b0e6_0_6_2_18: 0.21834437 # ball on top right court's side, and 0.01998301 BALL ON PLAY

- distance 1606b0e6_0_7_2_17: 0.02071127 # BALL ON PLAY

- distance 1606b0e6_0_8_2_24: 0.01237615 # ball confused with football boot

- distance 1606b0e6_0_9_2_21: 0.05960312 # BALL ON PLAY

- distance 1606b0e6_0_10_2_17: 0.16937768 # ball confused with head of person top bottom
- distance 1606b0e6_0_10_2_18: 0.1684855 # ball confused with head of person top bottom
- distance 1606b0e6_0_10_2_19: 0.16789392 # ball confused with head of person top bottom
- distance 1606b0e6_0_10_2_25: 0.04449379 # BALL ON PLAY
- distance 1606b0e6_0_10_2_17: 0.238802 # ball on top right court's side

- distance 1606b0e6_0_11_2_18: 0.23834602 # ball on top right court's side
- distance 1606b0e6_0_11_2_19: 0.16189149 # ball on top right court's side (same, person in crowd detected now)
- distance 1606b0e6_0_11_2_20: 0.06701084 # ball on top right court's side (same, another person detected now), and
0.0336005 BALL ON PLAY


### We set threshold to 0.065 for now.

In [ ]:
distance[0] > 0.08

### Code implemented

In [23]:
UNIQUE_INDEX = 0

In [24]:
ball_close_to_person = True
ball_person_threshold = 0.065 # AS STRICT AS I CAN BE, LOWER THAN 0.06 IS NOT OK
y_camera_threshold = 0.75

In [40]:
# videos = ['35bd9041_0', '3c993bd2_0']
# videos

['35bd9041_0', '3c993bd2_0']

### 1st cropping version

In [41]:
for video in videos:
  ! mkdir crops/$video
  # We iterate over every event, from 0 to 395
  for event in range(EVENTS_PER_VIDEO[video]):
  # for event in range(274):
    # We establish ball = False (in this event, ball wasn't yet recognized)
    ball = False
    # We should find for example f'1606b0e6_0_{frame}_' and establish that class_ equals what follows it

    # Sometimes fails for no reason and then works
    # it_worked = False
    # while not it_worked:
    #   try: 
    #     file = glob.glob(f'{substring}*.txt')[ANY_INDEX]
    #     substring = f'{FOLDER_LABELS}{video}/labels/{video}_{event}_'
    #     it_worked = True
    #   except IndexError:
    #     pass
    substring = f'{FOLDER_LABELS}{video}/labels/{video}_{event}_'
    file = glob.glob(f'{substring}*.txt')[ANY_INDEX]

    len_subs = len(substring)
    # print(substring)
    # print(len_subs)
    # print(file[len_subs])
    class_ = int(file[len_subs])

    event_id = f'{video}_{event}_{class_}'

    # We create a condition based on the class on how to iterate for the frames.
    # Challenges' frame between tolerances are respectively: 17 yes, 1 yes, 17 yes.
    # For play and throw-in are respectively: 9 no, 8 yes, 1 yes, 8 yes, 9 no.

    # if class_ == 0: # if it's a challenge
      
    # TO AVOID BALL FPs: we iterate over every frames, from 18 to 32
    for frame in range(FIRST_FRAME_USED, FRAMES_PER_EVENT - 2): # - 2 cause we need to use 2 more after the one detected
      frame_id = f'{event_id}_{frame}' # e.g. '1606b0e6_0_0_0_0'
      print(frame_id)
      label_txt = f'{frame_id}.txt'
      with open(f'{FOLDER_LABELS}{video}/labels/{label_txt}') as f:
        reader = f.read()
      detections = [i.split(' ') for i in reader.split('\n')][:-1] # [:-1] because last element is always "''"
      # counter = 0

      for detection in detections: # in a single frame

        if detection[CLASS_INDEX] == BALL_CLASS:
          # Ball is True for this event, it was detected in at least one of the frames.
          ball = True

          x = int(float(detection[X_INDEX]) * X_SIZE)
          y = int(float(detection[Y_INDEX]) * Y_SIZE) # 0.75 * 1028 = 771
          ymin = max(y - radius, 0) # 771 - 112 = 659
          ymax = min(y + radius, Y_SIZE) # 771 + 112 = 883
          xmin = max(x - radius, 0)
          xmax = min(x + radius, X_SIZE)
          # REVISAR WHY THERE ARE 55 EVENTS WITH SHAPES DIFFERENT THAN (224x224) IN THE WHOLE TRAIN DATA

          # TO AVOID BALL FPs: we throw away detections that make the crop next to a limit of the frame

          # CHANGED ymax == Y_SIZE to y > Y_SIZE * 0.75 TO BE MORE STRICT

          if ymin == 0 or y > Y_SIZE * y_camera_threshold or xmin == 0 or xmax == X_SIZE:
            ball = False

          # We make sure all crops are 224x224 (this won't apply if code above is implemented)
          # if ymin == 0:
          #   ymax = CROP_SIZE # = 224
          # elif ymax == Y_SIZE:
          #   ymin = Y_SIZE - CROP_SIZE # = 1024 - 224
          # if xmin == 0:
          #   xmax = CROP_SIZE # == 224 
          # elif xmax == X_SIZE:
          #   xmin = X_SIZE - CROP_SIZE # = 1928 - 224

          if ball_close_to_person:
            arr = np.array(detections)
            persons = arr[arr[:, 0] == PERSON_CLASS]
            xy_persons = persons[:, X_INDEX:Y_INDEX + 1].astype(float)
            xy_ball = np.array(detection[X_INDEX:Y_INDEX + 1]).astype(float)
            xy_closest_person = xy_persons[spatial.KDTree(xy_persons).query(xy_ball)[1]] # get closest person
            distance, index = spatial.KDTree(xy_persons).query(xy_ball) # get closest distance ball-person
            if distance > ball_person_threshold:
              ball = False

          if ball:
            for r in range(frame - 3, frame): # 3: amount of frames per event
              img = cv2.imread(f'{FOLDER_FRAMES}{video}/{event_id}_{r}.jpg')
              cropped_image = img[ymin:ymax, xmin:xmax] # We change from (x, y) to (y, x) to work with cv2_imshow
              cv2.imwrite(f'{POSITIVES_FOLDER}crops/{video}/{frame_id}_{r}cropped.jpg', cropped_image)
            break # breaks loop of detections in a single frame (so there are 3 per frame only)

# I ADDED THE FOLLOWING IF STATEMENT:
        # if ball:
        #   break

      if ball:      
        break # breaks loop of frame in frames per event (so there are 3 per event only)

      elif frame == FRAMES_PER_EVENT - 3: # We implement crop on main frame (index 17) based on closest two people
        frame_id = f'{event_id}_{FIRST_FRAME_USED - 1}' # -1 so we get main fraim (index 17)
        label_txt = f'{frame_id}.txt'
        with open(f'{FOLDER_LABELS}{video}/labels/{label_txt}') as f:
          reader = f.read()
        detections = [i.split(' ') for i in reader.split('\n')][:-1] # [:-1] because last element is always "''"
        arr = np.array(detections)
        persons = arr[arr[:, 0] == '0']
        xy_persons = persons[:, X_INDEX:Y_INDEX + 1].astype(float)
        dists = cdist(xy_persons, xy_persons)
        dists[dists == 0] = dists.max()
        arg1, arg2 = np.unravel_index(dists.argmin(), dists.shape)
        two_people = xy_persons[[arg1, arg2]]
        frame_center = np.mean(two_people, axis=0)
        x, y = frame_center
        x = int(x * X_SIZE)
        y = int(y * Y_SIZE)
        ymin = max(y - radius, 0)
        ymax = min(y + radius, Y_SIZE)
        xmin = max(x - radius, 0)
        xmax = min(x + radius, X_SIZE)

        for r in range(FIRST_FRAME_USED - 1, FIRST_FRAME_USED + 2): # here we use -1 and + 2 because we are certain it's from 17 (our main frame)
          img = cv2.imread(f'{FOLDER_FRAMES}{video}/{event_id}_{r}.jpg')
          cropped_image = img[ymin:ymax, xmin:xmax] # We change from (x, y) to (y, x) to work with cv2_imshow
          cv2.imwrite(f'{POSITIVES_FOLDER}crops/{video}/{frame_id}_{r}cropped.jpg', cropped_image)
        break


35bd9041_0_0_2_18
35bd9041_0_1_0_18
35bd9041_0_1_0_19
35bd9041_0_1_0_20
35bd9041_0_1_0_21
35bd9041_0_1_0_22
35bd9041_0_1_0_23
35bd9041_0_1_0_24
35bd9041_0_2_1_18
35bd9041_0_2_1_19
35bd9041_0_2_1_20
35bd9041_0_2_1_21
35bd9041_0_2_1_22
35bd9041_0_3_0_18
35bd9041_0_3_0_19
35bd9041_0_3_0_20
35bd9041_0_3_0_21
35bd9041_0_3_0_22
35bd9041_0_3_0_23
35bd9041_0_3_0_24
35bd9041_0_3_0_25
35bd9041_0_3_0_26
35bd9041_0_4_2_18
35bd9041_0_4_2_19
35bd9041_0_4_2_20
35bd9041_0_4_2_21
35bd9041_0_4_2_22
35bd9041_0_4_2_23
35bd9041_0_4_2_24
35bd9041_0_4_2_25
35bd9041_0_4_2_26
35bd9041_0_4_2_27
35bd9041_0_4_2_28
35bd9041_0_4_2_29
35bd9041_0_4_2_30
35bd9041_0_4_2_31
35bd9041_0_4_2_32
35bd9041_0_5_2_18
35bd9041_0_6_2_18
35bd9041_0_7_2_18
35bd9041_0_8_1_18
35bd9041_0_9_0_18
35bd9041_0_9_0_19
35bd9041_0_9_0_20
35bd9041_0_9_0_21
35bd9041_0_10_2_18
35bd9041_0_10_2_19
35bd9041_0_10_2_20
35bd9041_0_10_2_21
35bd9041_0_10_2_22
35bd9041_0_10_2_23
35bd9041_0_10_2_24
35bd9041_0_11_0_18
35bd9041_0_11_0_19
35bd9041_0_11_0_20


### 1st version, considering distance from balls detected in one frame. 

- Agarramos la pelota mas cercana a Y=0.5. La pelota real quizas no es detectada en el primer frame, asi que iteramos del frame 16 al 30 (15 frames) y agarramos la mas cercana.
- El problema es que quizas la pelota mas cercana (e.g. distancia de 0.05) al centro es la del frame 28, y nos serviria mas una pelota detectada con distancia 0.051 pero del frame 18, por estar el crop mas cerca del momento en el que la jugada sucede (frame 17).
- Por esto, agregamos una penalidad de distancia (quizas de 0.005) por cada frame que se incrementa. Entonces, en este ejemplo, una distancia de 0.05 en el frame 18 es menor que una distancia de 0.047 en el frame 28.


In [ ]:
for video in videos:
  ! mkdir crops/$video
  # We iterate over every event, from 0 to 395
  for event in range(EVENTS_PER_VIDEO[video]):
  # for event in range(274):
    # We establish ball = False (in this event, ball wasn't yet recognized)
    ball = False
    current_event_balls = list()
    # We should find for example f'1606b0e6_0_{frame}_' and establish that class_ equals what follows it

    substring = f'{FOLDER_LABELS}{video}/labels/{video}_{event}_'
    file = glob.glob(f'{substring}*.txt')[ANY_INDEX]

    len_subs = len(substring)
    class_ = int(file[len_subs])

    event_id = f'{video}_{event}_{class_}'

    # We create a condition based on the class on how to iterate for the frames.
    # Challenges' frame between tolerances are respectively: 17 yes, 1 yes, 17 yes.
    # For play and throw-in are respectively: 9 no, 8 yes, 1 yes, 8 yes, 9 no.

    # if class_ == 0: # if it's a challenge
      
    # TO AVOID BALL FPs: we iterate over every frames, from 18 to 32
    for frame in range(FIRST_FRAME_USED, FRAMES_PER_EVENT - 2): # - 2 cause we need to use 2 more after the one detected
      frame_id = f'{event_id}_{frame}' # e.g. '1606b0e6_0_0_0_0'
      print(frame_id)
      label_txt = f'{frame_id}.txt'
      with open(f'{FOLDER_LABELS}{video}/labels/{label_txt}') as f:
        reader = f.read()
      detections = [i.split(' ') for i in reader.split('\n')][:-1] # [:-1] because last element is always "''"
      # counter = 0

      for detection in detections: # in a single frame

        if detection[CLASS_INDEX] == BALL_CLASS:
          # Ball is True for this event, it was detected in at least one of the frames.
          ball = True

          x = int(float(detection[X_INDEX]) * X_SIZE)
          y = int(float(detection[Y_INDEX]) * Y_SIZE) # 0.75 * 1028 = 771
          ymin = max(y - radius, 0) # 771 - 112 = 659
          ymax = min(y + radius, Y_SIZE) # 771 + 112 = 883
          xmin = max(x - radius, 0)
          xmax = min(x + radius, X_SIZE)
          # REVISAR WHY THERE ARE 55 EVENTS WITH SHAPES DIFFERENT THAN (224x224) IN THE WHOLE TRAIN DATA

          # TO AVOID BALL FPs: we throw away detections that make the crop next to a limit of the frame

          # CHANGED ymax == Y_SIZE to y > Y_SIZE * 0.75 TO BE MORE STRICT

          if ymin == 0 or y > Y_SIZE * y_camera_threshold or xmin == 0 or xmax == X_SIZE:
            ball = False

          if ball_close_to_person:
            arr = np.array(detections)
            persons = arr[arr[:, 0] == PERSON_CLASS]
            xy_persons = persons[:, X_INDEX:Y_INDEX + 1].astype(float)
            xy_ball = np.array(detection[X_INDEX:Y_INDEX + 1]).astype(float)
            xy_closest_person = xy_persons[spatial.KDTree(xy_persons).query(xy_ball)[1]] # get closest person
            distance, index = spatial.KDTree(xy_persons).query(xy_ball) # get closest distance ball-person
            if distance > ball_person_threshold:
              ball = False

          if ball:
            distance_middle_screen = Y_SIZE / 2 - y
            current_event_balls.append([frame_id, ymin, ymax, xmin, xmax])

            for r in range(frame - 3, frame): # 3: amount of frames per event
              img = cv2.imread(f'{FOLDER_FRAMES}{video}/{event_id}_{r}.jpg')
              cropped_image = img[ymin:ymax, xmin:xmax] # We change from (x, y) to (y, x) to work with cv2_imshow
              cv2.imwrite(f'{POSITIVES_FOLDER}crops/{video}/{frame_id}_{r}cropped.jpg', cropped_image)
            break # breaks loop of detections in a single frame (so there are 3 per frame only)

      if ball:      
        break # breaks loop of frame in frames per event (so there are 3 per event only)

      elif frame == FRAMES_PER_EVENT - 3: # We implement crop on main frame (index 17) based on closest two people
        frame_id = f'{event_id}_{FIRST_FRAME_USED - 1}' # -1 so we get main fraim (index 17)
        label_txt = f'{frame_id}.txt'
        with open(f'{FOLDER_LABELS}{video}/labels/{label_txt}') as f:
          reader = f.read()
        detections = [i.split(' ') for i in reader.split('\n')][:-1] # [:-1] because last element is always "''"
        arr = np.array(detections)
        persons = arr[arr[:, 0] == '0']
        xy_persons = persons[:, X_INDEX:Y_INDEX + 1].astype(float)
        dists = cdist(xy_persons, xy_persons)
        dists[dists == 0] = dists.max()
        arg1, arg2 = np.unravel_index(dists.argmin(), dists.shape)
        two_people = xy_persons[[arg1, arg2]]
        frame_center = np.mean(two_people, axis=0)
        x, y = frame_center
        x = int(x * X_SIZE)
        y = int(y * Y_SIZE)
        ymin = max(y - radius, 0)
        ymax = min(y + radius, Y_SIZE)
        xmin = max(x - radius, 0)
        xmax = min(x + radius, X_SIZE)

        for r in range(FIRST_FRAME_USED - 1, FIRST_FRAME_USED + 2): # here we use -1 and + 2 because we are certain it's from 17 (our main frame)
          img = cv2.imread(f'{FOLDER_FRAMES}{video}/{event_id}_{r}.jpg')
          cropped_image = img[ymin:ymax, xmin:xmax] # We change from (x, y) to (y, x) to work with cv2_imshow
          cv2.imwrite(f'{POSITIVES_FOLDER}crops/{video}/{frame_id}_{r}cropped.jpg', cropped_image)
        break


### 2nd cropping version:

1. We are going to implement cropping in every detection of balls from index frame 10 to 24 (15 in total, being index 17 the main frame: 7 before, 1 main frame, and 7 after)
2. Then on each one of them, e.g. frame 10, apply cropping based of the position of the ball from 10 to 12 (with all balls detected on frame 10). Then go to frame 11, perform cropping on 11 to 13, and so on until frame 24 to 26.
3. After that, start checking movement from frame 17 (including 17 to 19), if there isn't, go to frame 18 (including 18 to 20) and check again, AND THINK OF HOW TO CONTINUE FROM MAIN THREE FRAMES TO LESS RELEVANT ONES.
4. What to do if in a single frame more than one ball is detected "with movement"? What if in all 15 gray-scaled frames there is no ball detected "which movement"? Think about it.

In [ ]:
POSITIVES_FOLDER = '/content/drive/MyDrive/Bundesliga/PositiveFrames/'
os.chdir(POSITIVES_FOLDER)

In [ ]:
! pwd

/content/drive/MyDrive/Bundesliga/PositiveFrames


In [ ]:
# os.chdir('/content/drive/MyDrive/Bundesliga/')

In [ ]:
# ! ls crops

In [236]:
! ls crops/'1606b0e6_0'  | wc -l

1188


In [ ]:
! ls crops/'3c993bd2_1'  | wc -l

1128


In [ ]:
! pwd

/content/drive/MyDrive/Bundesliga/PositiveFrames


First time checked:

In [ ]:
for video_id in video_ids:
  print(video_id)
  ! ls frames_ms2/$video_id | wc -l

1606b0e6_0
13860
1606b0e6_1
2426
cfbe2e94_0
3040
cfbe2e94_0
3040
ecf251d4_0
648
3c993bd2_0
659
3c993bd2_1
16036
4ffd5986_0
3050
9a97dae4_1
3091
35bd9041_0
636
35bd9041_1
587
407c5a9e_1
670


Second time checked:

In [ ]:
for video_id in video_ids:
  print(video_id)
  ! ls $FOLDER_FRAMES/$video_id | wc -l

1606b0e6_0
13860
1606b0e6_1
3940
cfbe2e94_0
4549
cfbe2e94_1
1378
ecf251d4_0
9743
3c993bd2_0
1688
3c993bd2_1
16036
4ffd5986_0
4632
9a97dae4_1
5554
35bd9041_0
1466
35bd9041_1
1377
407c5a9e_1
10498


Third time checked:

In [ ]:
for video_id in video_ids:
  print(video_id)
  ! ls $FOLDER_FRAMES/$video_id | wc -l

1606b0e6_0
13860
1606b0e6_1
4374
cfbe2e94_0
5003
cfbe2e94_1
2888
ecf251d4_0
13510
3c993bd2_0
3364
3c993bd2_1
16036
4ffd5986_0
5090
9a97dae4_1
10185
35bd9041_0
3081
35bd9041_1
2923
407c5a9e_1
12250


Fourth time checked - 12/11/22 1.14 am:

In [ ]:
for video_id in video_ids:
  print(video_id)
  ! ls $FOLDER_FRAMES/$video_id | wc -l

1606b0e6_0
13860
1606b0e6_1
7823
cfbe2e94_0
8637
cfbe2e94_1
6383
ecf251d4_0
13510
3c993bd2_0
7248
3c993bd2_1
16036
4ffd5986_0
8661
9a97dae4_1
10185
35bd9041_0
7049
35bd9041_1
6522
407c5a9e_1
12250


Fifth time checked - 12/11/22 8.31 am:

In [24]:
for video_id in video_ids:
  print(video_id)
  ! ls $FOLDER_FRAMES/$video_id | wc -l

1606b0e6_0
13860
1606b0e6_1
9594
cfbe2e94_0
10675
cfbe2e94_1
9975
ecf251d4_0
13510
3c993bd2_0
12247
3c993bd2_1
16036
4ffd5986_0
10570
9a97dae4_1
10185
35bd9041_0
10633
35bd9041_1
12565
407c5a9e_1
12250


Sixth time checked - 12/11/22 22.19 h (after Yosef deleted duplicate files):

In [25]:
for video_id in video_ids:
  print(video_id)
  ! ls $FOLDER_FRAMES/$video_id | wc -l

1606b0e6_0
13860
1606b0e6_1
9594
cfbe2e94_0
10675
cfbe2e94_1
9975
ecf251d4_0
13510
3c993bd2_0
12247
3c993bd2_1
13160
4ffd5986_0
10570
9a97dae4_1
10185
35bd9041_0
10633
35bd9041_1
12565
407c5a9e_1
12250


In [262]:
videos

['1606b0e6_0',
 'cfbe2e94_0',
 'cfbe2e94_1',
 'ecf251d4_0',
 '3c993bd2_1',
 '4ffd5986_0',
 '9a97dae4_1',
 '407c5a9e_1']

Checking amount of files per video with frames cropped:

In [43]:
for video in video_ids:
  print(video)
  ! ls crops/$video | wc -l

1606b0e6_0
1188
1606b0e6_1
822
cfbe2e94_0
915
cfbe2e94_1
855
ecf251d4_0
1158
3c993bd2_0
822
3c993bd2_1
1128
4ffd5986_0
906
9a97dae4_1
873
35bd9041_0
822
35bd9041_1
1077
407c5a9e_1
1050


In [45]:
video_ids

['1606b0e6_0',
 '1606b0e6_1',
 'cfbe2e94_0',
 'cfbe2e94_1',
 'ecf251d4_0',
 '3c993bd2_0',
 '3c993bd2_1',
 '4ffd5986_0',
 '9a97dae4_1',
 '35bd9041_0',
 '35bd9041_1',
 '407c5a9e_1']

In [44]:
for video in video_ids:
  print(video)
  ! ls frames_gray/$video | wc -l

1606b0e6_0
396
1606b0e6_1
274
cfbe2e94_0
305
cfbe2e94_1
285
ecf251d4_0
386
3c993bd2_0
274
3c993bd2_1
376
4ffd5986_0
302
9a97dae4_1
291
35bd9041_0
274
35bd9041_1
359
407c5a9e_1
350
